In [1]:
# Copyright 2025 L3Harris Technologies, Inc.
#
#    Licensed under the Apache License, Version 2.0 (the "License");
#    you may not use this file except in compliance with the License.
#    You may obtain a copy of the License at
#
#        http://www.apache.org/licenses/LICENSE-2.0
#
#    Unless required by applicable law or agreed to in writing, software
#    distributed under the License is distributed on an "AS IS" BASIS,
#    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#    See the License for the specific language governing permissions and
#    limitations under the License.
#

In [2]:
import numpy as np 
import sympy

from copy import copy

import matplotlib.pyplot as plt

from pprint import pprint

import pandas as pd

In [3]:
def print_matrix_info(matrix_name, return_val: bool=False):
    A = globals()[matrix_name]
    
    A_tol = copy(A)
    A_tol[np.abs(A) < 1e-14] = 0.0 # truncate really small values to zero.
    
    num_elements = np.size(A_tol)
    num_nonzero_elements = np.count_nonzero(A_tol)
    unique_elements = np.unique(A_tol)
    num_unique_elements = len(unique_elements)
    if 0.0 in unique_elements:
        num_unique_nonzero_elements = num_unique_elements - 1 # minus 1 because not including zero!
    else:
        num_unique_nonzero_elements = num_unique_elements

    description = (f"{matrix_name}, shape={A.shape}, rank={np.linalg.matrix_rank(A)}, num_elements={num_elements}, num_nonzero={num_nonzero_elements}, num_unique={num_unique_elements}, num_unique_nonzero={num_unique_nonzero_elements}")
    if return_val:
        return description
    else: 
        print(description)


In [4]:
# Load the F-matrices that were created in `bounding_A_norm.ipynb`

F1_matrix = np.load("F_1_matrix.npz")["F_1_matrix"]
F2_matrix = np.load("F_2_matrix.npz")["F_2_matrix"]
F3_matrix = np.load("F_3_matrix.npz")["F_3_matrix"]

print("WARNING!!!  These matrices have not been scaled by the tau parameter yet!")
print_matrix_info(matrix_name="F1_matrix")
print_matrix_info(matrix_name="F2_matrix")
print_matrix_info(matrix_name="F3_matrix")

WARNING!!!  These matrices have not been scaled by the tau parameter yet!
F1_matrix, shape=(27, 27), rank=23, num_elements=729, num_nonzero=729, num_unique=14, num_unique_nonzero=14
F2_matrix, shape=(27, 729), rank=6, num_elements=19683, num_nonzero=15180, num_unique=43, num_unique_nonzero=42
F3_matrix, shape=(27, 19683), rank=6, num_elements=531441, num_nonzero=409860, num_unique=43, num_unique_nonzero=42


In [5]:
tau = sympy.Symbol("tau")

F1_norm =  (1/tau)*(np.linalg.norm(F1_matrix, ord=np.inf))
F2_norm =  (1/tau)*(np.linalg.norm(F2_matrix, ord=np.inf))
F3_norm =  (1/tau)*(np.linalg.norm(F3_matrix, ord=np.inf))

print("All norms are the `infinity` norm")
print(f"F1_norm: {F1_norm}")
print(f"F2_norm: {F2_norm}")
print(f"F3_norm: {F3_norm}")

tau_val = 0.6
F1_norm_val =  F1_norm.evalf(subs={tau: tau_val})
F2_norm_val =  F2_norm.evalf(subs={tau: tau_val})
F3_norm_val =  F3_norm.evalf(subs={tau: tau_val})


S_norm_val = 2
S_norm = S_norm_val

print(f"\nIf tau={tau_val}, then...")
print(f"\tF1_norm: {F1_norm_val}")
print(f"\tF2_norm: {F2_norm_val}")
print(f"\tF3_norm: {F3_norm_val}")

print(f"\n\tS_norm: {S_norm_val}")


All norms are the `infinity` norm
F1_norm: 8.40740740740741/tau
F2_norm: 565.333333333333/tau
F3_norm: 7632.0/tau

If tau=0.6, then...
	F1_norm: 14.0123456790123
	F2_norm: 942.222222222223
	F3_norm: 12720.0000000000

	S_norm: 2


In [6]:
print(f"Reynolds numbers:")
Re = [
    1e1,
    1e2,
    1e3,
    1e4,
    1e5,
    1e6,
    1e7,
    1e8
]
print(Re)



print(f"\nTime discretization from table 8 (sphere instances):")
delta_t = [
    3.323363e2,
    3.323363e0,
    3.323363e-2,
    3.323363e-4,
    3.323363e-6,
    3.323363e-8,
    3.323363e-10,
    3.323363e-12
]
print(delta_t)

Reynolds numbers:
[10.0, 100.0, 1000.0, 10000.0, 100000.0, 1000000.0, 10000000.0, 100000000.0]

Time discretization from table 8 (sphere instances):
[332.3363, 3.323363, 0.03323363, 0.0003323363, 3.323363e-06, 3.323363e-08, 3.323363e-10, 3.323363e-12]


### From the Appendix: Time domain convergence of the Carleman linearized system:

For some evolution time $T_c$ (in lattice units), as the truncation order $K$ is increased, $K \longrightarrow \infty$, the error from the truncated Carleman linearized system becomes arbitrarily small.  Theorem I.3 establishes a lower and upper bound on $T_c$ as

$ \frac{1}{2\|\phi_0\| \left( \| F_2\| + \|F_3\|\right) + \|S\| + \|F_1\| + \|F_2 \| } \leq T_c \le \frac{1}{2\|\phi_0\| \left( \|F_2\| + \|F_3\| \right)} $,

where all norms are the $\infty$-norm and $\phi$ represents the variables of the Carleman linearized system as 
$\phi = f^{\otimes 1} \oplus f^{\otimes 2} \oplus f^{\otimes 3} \oplus ... \oplus f^{\otimes K}$,
and $\phi_0 = \phi(t=0)$ is some initial solution.

We estimate $\| \phi_0 \|_\infty$ as follows.  In the initial conditions, the ambient flow is in the $x$-direction, so $\mathbf{u} = (u, 0, 0)$.    






In [7]:
phi0 = sympy.Symbol("phi0")

Tc_lower_bound = 1/(2*phi0*(F2_norm + F3_norm) + S_norm + F1_norm + F2_norm)
Tc_upper_bound = 1/(2*phi0*(F2_norm + F3_norm))

print(f"Tc_lower_bound: {Tc_lower_bound}")
print(f"Tc_upper_bound: {Tc_upper_bound}")


phi0_val = 0.29580246914876046
Tc_lower_bound_val = Tc_lower_bound.evalf(subs={tau:tau_val, phi0:phi0_val})
Tc_upper_bound_val = Tc_upper_bound.evalf(subs={tau:tau_val, phi0:phi0_val})

print(f"\nIf tau={tau_val} and phi0={phi0_val}, then...")
print(f"\tTc_lower_bound: {Tc_lower_bound_val}")
print(f"\tTc_upper_bound: {Tc_upper_bound_val}")


print(f"\n(See the initial_flow_vector.ipynb for a justification of the value for phi0)")


Tc_lower_bound: 1/(16394.6666666667*phi0/tau + 2 + 573.740740740741/tau)
Tc_upper_bound: 6.09954456733897e-5*tau/phi0

If tau=0.6 and phi0=0.29580246914876046, then...
	Tc_lower_bound: 0.000110608791085279
	Tc_upper_bound: 0.000123721980784511

(See the initial_flow_vector.ipynb for a justification of the value for phi0)


## Commentary:

As the units for $T_c$ are lattice units and the upper bound is below 1, this implies that we cannot guarantee convergence for even one lattice time step.  The error may actually be Ok, but we cannot guarantee it.

We proceed to calculate bounds on the physical evolution time $T^*$ which is related to the lattice evolution time $T$ as $\frac{T^*}{\Delta t} = T$.

In [8]:
data = {
    "Re":Re,
    "$\Delta t$": delta_t,
    "$T^*$ lower bound": [0.0]*8,
    "$T^*$ upper bound": [0.0]*8    
}
df = pd.DataFrame(data)

for j in range(8):
    Delta_t = df.loc[j, "$\Delta t$" ] 
    df.loc[j, "$T^*$ lower bound"] = Tc_lower_bound_val*Delta_t
    df.loc[j, "$T^*$ upper bound"] = Tc_upper_bound_val*Delta_t
    
display(df)

/tmp/ipykernel_265940/3954203516.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0367593163767546' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[j, "$T^*$ lower bound"] = Tc_lower_bound_val*Delta_t
/tmp/ipykernel_265940/3954203516.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0411173053225955' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[j, "$T^*$ upper bound"] = Tc_upper_bound_val*Delta_t


,Re,$\Delta t$,$T^*$ lower bound,$T^*$ upper bound
0,10.0,3.323363e+02,0.0367593163767546,0.0411173053225955
1,100.0,3.323363e+00,0.000367593163767546,0.000411173053225955
2,1000.0,3.323363e-02,3.67593163767546e-6,4.11173053225955e-6
3,10000.0,3.323363e-04,3.67593163767546e-8,4.11173053225955e-8
4,100000.0,3.323363e-06,3.67593163767546e-10,4.11173053225955e-10
5,1000000.0,3.323363e-08,3.67593163767546e-12,4.11173053225955e-12
6,10000000.0,3.323363e-10,3.67593163767546e-14,4.11173053225955e-14
7,100000000.0,3.323363e-12,3.67593163767546e-16,4.11173053225955e-16
